In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from datetime import datetime

import matplotlib.pyplot as plt

In [2]:
year_path = [(2018, '/datadrive/timelapse_images/')]

In [3]:
df = pd.DataFrame()
for y, _ in year_path:
    path = '/datadrive/timeseries_derived_data_products/MH11_resistivity_rock_{}.csv'.format(y)
    df = pd.concat((df, pd.read_csv(path)), axis=0)

In [4]:
df2 = pd.DataFrame()
for y, _ in year_path:
    path2 = '/datadrive/timeseries_derived_data_products/MH25_vaisalawxt520prec_{}.csv'.format(y)
    df2 = pd.concat((df2, pd.read_csv(path2)), axis=0)

In [5]:
df3 = pd.DataFrame()
for y, _ in year_path:
    path3 = '/datadrive/timeseries_derived_data_products/MH25_vaisalawxt520windpth_2017.csv'.format(y)
    df3 = pd.concat((df3, pd.read_csv(path3)), axis=0)

In [6]:
df.head()

,time,position [],device_id [],resistivity_10cm [Mohm],resistivity_35cm [Mohm],resistivity_60cm [Mohm],resistivity_85cm [Mohm],resistivity_10_35cm [Mohm],resistivity_35_60cm [Mohm],resistivity_60_85cm [Mohm],resistivity_10_60cm [Mohm],resistivity_35_85cm [Mohm],resistivity_10_85cm [Mohm],ref1 [°C],ref2 [°C],ref3 [°C],ref4 [°C],ref5 [°C]
0,2018-01-01T00:00:00Z,11.0,2096.0,1465.6452,862.3770,156.1253,24.3910,247.5946,41.3651,36.8923,1716.4263,333.9713,1780.0550,1.8783,-9.7979,1.0,1.8803,-9.7969
1,2018-01-01T01:00:00Z,11.0,2096.0,1486.2808,873.0369,158.4006,24.3173,250.2695,41.3379,36.8208,1730.2799,337.3985,1778.4165,1.8783,-9.7978,1.0,1.8803,-9.7971
2,2018-01-01T02:00:00Z,11.0,2096.0,1487.4257,876.1179,159.6845,24.2089,252.8374,41.2581,36.6818,1728.7300,340.5199,1778.4711,1.8785,-9.7976,1.0,1.8806,-9.7968
3,2018-01-01T03:00:00Z,11.0,2096.0,1487.3720,888.8951,162.2301,24.1631,255.4060,41.2239,36.5137,1751.9790,345.2815,1789.8859,1.8787,-9.7974,1.0,1.8805,-9.7968
4,2018-01-01T04:00:00Z,11.0,2096.0,1486.3813,890.0825,162.6218,24.1580,259.7650,41.3469,36.4391,1773.7597,350.3218,1825.9325,1.8778,-9.7982,1.0,1.8797,-9.7977


In [7]:
def interpolate_to_timestamps(df, time_stamps):
    df = df.resample('4T').interpolate()
    ind = [df.index.get_loc(tim, method='nearest') for tim in time_stamps.time]
    return df.iloc[ind, :]

In [8]:
def extract_time_stamps(year_path):
    """
    input: [(year, root_dir), ...]
    """
    dfs = pd.DataFrame()
    for y, root_dir in year_path:
        path_dict = {}
        for filename in glob.iglob(root_dir + '**/'+str(y)+'*/*', recursive=True):
            di, filname = os.path.split(filename)
            ddi, ydi = os.path.split(di)
            path_dict[filname] = extract_time(filname)
        df = pd.DataFrame({'time':list(path_dict.values()), 'filename': list(path_dict.keys())}, )
        dfs = pd.concat((dfs, df), axis=0)
    return dfs

In [9]:
def extract_time(filname):
    return datetime.strptime(filname, '%Y%m%d_%H%M%S.JPG')

In [10]:
def extract_summer_days(time_stamps):
    start = pd.Timestamp(datetime(year=2000, month=1, day=1, hour=8)).time()
    end = pd.Timestamp(datetime(year=2000, month=1, day=1, hour=20)).time()
    time_stamps_day = time_stamps[np.logical_and(time_stamps['time'].dt.time >= start , 
                                                 time_stamps['time'].dt.time <= end)]
    
    june = pd.Timestamp(datetime(year=2000, month=5, day=1, hour=8)).month
    august = pd.Timestamp(datetime(year=2000, month=8, day=1, hour=8)).month

    time_stamps_summerday = time_stamps_day[np.logical_and(time_stamps['time'].dt.month < august , 
                                                           time_stamps['time'].dt.month >= june)]
    
    return time_stamps_summerday


In [11]:
#time_stamps = pd.read_pickle('pd_time_stamps.pkl')
#time_stamps.head()
time_stamps = extract_time_stamps(year_path)
summer_days = extract_summer_days(time_stamps)
time_stamps.head()

/home/ivan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


,time,filename
0,2018-02-23 20:00:08,20180223_200008.JPG
1,2018-02-23 02:00:07,20180223_020007.JPG
2,2018-02-23 14:00:07,20180223_140007.JPG
3,2018-02-23 16:00:07,20180223_160007.JPG
4,2018-02-23 12:00:07,20180223_120007.JPG


In [12]:
#summer_days = pd.read_pickle('pd_time_stamps_summerdays.pkl')

In [13]:
df = df.set_index(pd.DatetimeIndex(df.loc[:, 'time']))
df_interp = interpolate_to_timestamps(df, summer_days)

/home/ivan/miniconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:3071: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  target = np.asarray(target)


In [14]:
df2 = df2.set_index(pd.DatetimeIndex(df2.loc[:, 'time']))
df2_interp = interpolate_to_timestamps(df2, summer_days)

In [15]:
df3 = df3.set_index(pd.DatetimeIndex(df3.loc[:, 'time']))
df3_interp = interpolate_to_timestamps(df3, summer_days)

In [16]:
#df_interp = df_interp.loc[~df_interp.index.duplicated(keep='first')]
#df2_interp = df2_interp.loc[~df2_interp.index.duplicated(keep='first')]
#df3_interp = df3_interp.loc[~df3_interp.index.duplicated(keep='first')]

#dataset = pd.concat((df_interp, df2_interp, df3_interp), axis=1)
df_interp['path'] = summer_days.filename.values
#dataset = dataset.rename(columns={'rain_intensity [mm/h]': 'rain_intensity'})

In [17]:
df_interp['label_thresh_rest10'] = df_interp.loc[:, 'resistivity_10cm [Mohm]'] > 50
df_interp['rain_label'] = df2_interp.loc[:, 'rain_intensity [mm/h]'] > 1

In [18]:
df_interp.query('rain_label == False').loc[:, ['path', 'label_thresh_rest10']].to_csv('labels_rain_resist.csv', header=False)
df_interp.loc[:, ['path', 'label_thresh_rest10']].to_csv('labels.csv', header=False)

/home/ivan/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:456: FutureWarning: The default of the 'keep_tz' keyword in DatetimeIndex.to_series will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  s = level_values.to_series()
/home/ivan/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:464: FutureWarning: The default of the 'keep_tz' keyword in DatetimeIndex.to_series will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  dindex = axis_index.to_series()


In [19]:
summer_days.filename

33      20180515_080010.JPG
35      20180515_180007.JPG
37      20180515_120007.JPG
39      20180515_100011.JPG
40      20180515_160007.JPG
               ...         
3835    20180726_160007.JPG
3836    20180726_100011.JPG
3837    20180726_080011.JPG
3838    20180726_120007.JPG
3839    20180726_140008.JPG
Name: filename, Length: 552, dtype: object

In [21]:
import io
import csv
import shutil
img_folder = '/datadrive/timelapse_images'
img_folder_test = '/datadrive/timelapse_images_test'

with open('/datadrive/labels.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        print(row)
        img_name = row[1]
        label = int(row[2]=='True')
        month_folder = row[0][:10]
        #print(month_folder)
        #print(img_name,label)
        print(os.path.join(img_folder,month_folder,img_name))
        shutil.copyfile(os.path.join(img_folder,month_folder,img_name),os.path.join(img_folder_test,str(label),img_name))

['2018-05-15 08:00:00+00:00', '20180515_080010.JPG', 'True']
/datadrive/timelapse_images/2018-05-15/20180515_080010.JPG
['2018-05-15 18:00:00+00:00', '20180515_180007.JPG', 'True']
/datadrive/timelapse_images/2018-05-15/20180515_180007.JPG
['2018-05-15 12:00:00+00:00', '20180515_120007.JPG', 'True']
/datadrive/timelapse_images/2018-05-15/20180515_120007.JPG
['2018-05-15 10:00:00+00:00', '20180515_100011.JPG', 'True']
/datadrive/timelapse_images/2018-05-15/20180515_100011.JPG
['2018-05-15 16:00:00+00:00', '20180515_160007.JPG', 'True']
/datadrive/timelapse_images/2018-05-15/20180515_160007.JPG
['2018-05-15 14:00:00+00:00', '20180515_140007.JPG', 'True']
/datadrive/timelapse_images/2018-05-15/20180515_140007.JPG
['2018-06-24 16:00:00+00:00', '20180624_160008.JPG', 'True']
/datadrive/timelapse_images/2018-06-24/20180624_160008.JPG
['2018-06-24 10:00:00+00:00', '20180624_100011.JPG', 'True']
/datadrive/timelapse_images/2018-06-24/20180624_100011.JPG
['2018-06-24 12:00:00+00:00', '20180624_